# Object Detection with Conventional Algorithms
## Import libraries
Install numpy, yaml, matplotlib

**!** starts a command

In [ ]:
!pip install numpy pyyaml matplotlib

In [ ]:
import cv2 as cv
import numpy as np
from typing import List, Tuple
import yaml
import math
from utils import *

## Define the Armor class

In [ ]:
class Armor:
    def __init__(self, left: RotatedRect, right: RotatedRect):
        left = RotatedRect(left.center, (left.size[0], left.size[1] * 2), left.angle)
        right = RotatedRect(right.center, (right.size[0], right.size[1] * 2), right.angle)
        left_points = left.points()
        right_points = right.points()
        self.points = [left_points[0], left_points[1], right_points[2], right_points[3]]
        self.id = -1 # Unused in this class

    # Prompt: Complete the area function by calculating the area from the four vertices in self.points in class Armor
    # Hint: Use the shoelace formula to calculate the area of a polygon
    # https://en.wikipedia.org/wiki/Shoelace_formula
    def area(self):
        pass
    
    # Prompt: Write the center function to get the center of mass
    def center(self):
        pass

    def __str__(self):
        return "Armor: " + str(self.points)
    
    def __lt__(self, other):
        return self.area() < other.area()

## The Detector class
It contains many parameters saved in a yaml file.

In [ ]:
class Detector:
    # Prompt: Load these parameters from parameters.yaml and load each param into a variable like: armor_max_aspect_ratio = params['armor_max_aspect_ratio']
    def __init__(self, yaml_path: str):
        pass

### Overall program structure
- Separate the colors you wish to detect
- Binarize the image with a bright threshold
- Get the contours of the image
- Filter the contours, Remove what you don't want, Get the desired light indicators
- Match the light indicators to get armor

In [ ]:
# Prompt: Get a grayscale image from the difference between the red channel and the blue channel using OpenCV
# Hint: cv.subtract
# https://docs.opencv.org/4.x/d2/de8/group__core__array.html#ga10ac1bfbef7d6d2f1c8b4f7d7cde61b0
@add_to_class(Detector)
def _separateColors(self, img: np.ndarray, color: str) -> np.ndarray:
    if color == 'r':
        gray_img = ## Your code here!!!
    else:
        gray_img = ##  Your code here!!!

    return gray_img

In [ ]:
detector = Detector("parameters.yaml")
image = cv.imread("pictures/APC_0037.jpg")
gray_image = detector._separateColors(image, "r")
showImageOnNotebook(gray_image)

In [ ]:
# Prompt: Binarize the image using the brightness_threshold parameter and dilate the bright parts
# Ellipse: Change it so that the kernel is an ellipse
# Hint: cv.threshold, cv.dilate
@add_to_class(Detector)
def _binarization(self, gray_img: np.ndarray, brightness_threshold: int) -> np.ndarray:
    # Your code here!!!
    pass
    return dilated_img

In [ ]:
bin_image = detector._binarization(gray_image, 100)
showImageOnNotebook(bin_image)

In [ ]:
# Prompt: Find the contours in all the bright parts in the dilated image using OpenCV.
# Additional: Only show the added/modified code
# hint: cv.findContours 
@add_to_class(Detector)
def _getContours(self, img: np.ndarray) -> List[List[np.ndarray]]:
    # Your code here!!!
    pass
    return light_contours

In [ ]:
contours = detector._getContours(bin_image)
contour_image = drawAllContours(image, contours)
showImageOnNotebook(contour_image)

In [ ]:
@add_to_class(Detector)
def _filterContours(self, light_contours: List[List[np.ndarray]]) -> Tuple[List[List[np.ndarray]], List[RotatedRect]]:
    remaining_contours = []
    light_info = []
    for contour in light_contours:
        pass
    return remaining_contours, light_info

In [ ]:
@add_to_class(Detector)
def _adjustRect(self, rec: RotatedRect) -> RotatedRect:
    center, size, angle = rec.center, rec.size, rec.angle
    width, height = size

    while angle >= 90.0:
        angle -= 180.0
    while angle < -90.0:
        angle += 180.0

    if angle >= 45.0:
        width, height = height, width
        angle -= 90.0
    elif angle < -45.0:
        width, height = height, width
        angle += 90.0

    size = (width, height)

    return RotatedRect(center, size, angle)


In [ ]:
contours, light_info = detector._filterContours(contours)
filtered_contour_image = drawAllContours(image, contours)
showImageOnNotebook(filtered_contour_image)

In [ ]:
@add_to_class(Detector)
def _matchLightIndicators(self, light_info: List[RotatedRect]) -> List[Armor]:
    armors = []

    # Prompt: Sort a list of RotatedRect objects by the x (first) coordinate of the center
    
    
    for i in range(len(light_info)):
        left = light_info[i]
        for j in range(i + 1, len(light_info)):
            right = light_info[j]

            pass

    return armors

In [ ]:
armors = detector._matchLightIndicators(light_info)
annotated_image = drawArmors(image, armors)
showImageOnNotebook(annotated_image)

In [ ]:
@add_to_class(Detector)
def analyze(self, img: np.ndarray, color: str) -> List[Armor]:
    img = self._separateColors(img, color)
    img = self._binarization(img, self.brightness_threshold)
    contours = self._getContours(img)
    contours, light_info = self._filterContours(contours)
    armors = self._matchLightIndicators(light_info)

    return armors

In [ ]:
detector = Detector("parameters.yaml")
image = cv.imread("pictures/APC_0037.jpg")
armors = detector.analyze(image, "r")
print('Found', len(armors), 'armors.')
if len(armors) > 0:
    armors.sort()
    print('The first armor is', armors[0])
annotated_image = drawArmors(image, armors)
showImageOnNotebook(annotated_image)